In [ ]:
!pip install -qq gradio

In [ ]:
import gradio as gr
import numpy as np

from keras.models import load_model  # TensorFlow is required for Keras to work
from PIL import Image, ImageOps  # Install pillow instead of PIL

In [ ]:
if 'google.colab' in str(get_ipython()):
    # support save files in drive
    from google.colab import drive
    drive.mount('/content/drive')

In [ ]:
PROJECT_DIR = '/content/drive/MyDrive/Colab Notebooks/teachable-gradio'
PROJECT_NAME = 'sample_project'
TEACHABLE_MODEL_DIR = f'{PROJECT_DIR}/{PROJECT_NAME}/converted_keras'
SHAPE = (224, 224)

# NOTE:
#   Teachable Website: https://teachablemachine.withgoogle.com/train/image

In [ ]:
# load the model and label
model = load_model(f"{TEACHABLE_MODEL_DIR}/keras_model.h5", compile=False)
class_names = open(f"{TEACHABLE_MODEL_DIR}/labels.txt", "r").readlines()


def pre_process_image(image):
    # resizing the image to be at least 224x224 and then cropping from the center
    image = ImageOps.fit(image, SHAPE, Image.LANCZOS)

    # turn the image into a numpy array
    image_array = np.asarray(image)

    # normalize the image
    normalized_image_array = (image_array.astype(np.float32) / 127.5) - 1

    # reshape the image to batch format
    reshaped_image_array = normalized_image_array.reshape((-1, SHAPE[0], SHAPE[1], 3))

    return reshaped_image_array


def classify_image(image):
    image = pre_process_image(image)
    prediction = model.predict(image)
    confidences = {class_names[i]: float(prediction[0][i]) for i in range(len(class_names))}
    return confidences

In [ ]:
gr.Interface(fn=classify_image,
             inputs=gr.Image(shape=SHAPE, type='pil'),
             outputs=gr.Label(num_top_classes=2)).launch(quiet=True)

# NOTE:
#   use .launch(debug=True) to debug your gradio interface